# Air Sac Tracking Prototype code
wim.pouw@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [9]:
# import the necessary packages
import numpy as np
import argparse
import cv2
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np


#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

In [10]:
videofolder= '../Video/'
videofilename = 'sample.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second


#output video
out = cv2.VideoWriter('./Output/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))

#initialize iterator
i=0

#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    ############################detect circles   
    output=frame.copy()
    # apply GuassianBlur to reduce noise. medianBlur is also added for smoothening, reducing noise.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(11,11),0)
    gray = cv2.medianBlur(gray, 19)
    #thresholded edge contouring
    hist = cv2.equalizeHist(gray)
    gamma = 2
    invGamma = 1/gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    gamm = cv2.LUT(hist, table, hist)
    submitted = cv2.Canny(gamm, 20, 10)
    submitted = cv2.GaussianBlur(submitted,(7,7),0)
    submitted = cv2.threshold(submitted, 20, 255, cv2.THRESH_BINARY)[1]
    submitted = cv2.dilate(submitted, None, iterations=9)  
    submitted = cv2.erode(submitted, None, iterations=7)
    
    #track demicircles    
    circles = cv2.HoughCircles(submitted, cv2.HOUGH_GRADIENT, 1, 10000, param1=1,param2=10, minRadius=5, maxRadius=250)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(output, (x, y), r, (255, 255, 0), 2)
            cv2.circle(submitted, (x, y), r, (200, 0, 0), 2)
            print(x,y,r)
    cv2.waitKey(1)
    out.write(output)
    cv2.imshow("output", submitted)
    i=i+1

        
out.release()
cap.release()
cv2.destroyAllWindows()

618 554 130
640 438 240
626 554 116
1400 738 230
1390 746 219
1354 664 206
1394 744 223
660 538 106
1360 674 212
164 576 156
1566 702 245
176 582 168
1392 652 232
1404 628 225
680 500 129
678 496 128
694 496 140
1366 686 210
704 508 145
698 510 148
700 522 146
690 556 119
684 558 123
686 558 128
686 548 140
686 548 146
688 554 148
688 558 151
688 560 155
688 564 159
688 572 159
688 582 157
686 582 163
684 590 164
684 588 171
684 590 176
684 598 175
682 602 177
682 604 178
682 606 181
682 606 184
680 612 183
680 608 188
680 614 188
680 614 190
680 616 190
678 616 192
678 620 191
680 616 194
680 618 193
680 618 195
682 616 197
682 620 195
682 622 195
682 620 196
682 620 197
682 620 197
682 624 194
682 622 195
686 618 200
682 620 195
682 620 194
680 620 193
682 616 194
680 624 188
680 622 189
682 618 192
682 620 189
682 624 185
684 622 187
690 614 195
684 620 185
686 620 184
688 616 186
688 616 184
688 612 184
688 612 184
688 616 180
688 614 180
688 610 183
688 612 180
688 618 174
696 602

884 614 184
890 624 176
884 614 186
888 616 185
1456 494 249
890 618 183
888 610 190
894 618 183
900 646 156
896 620 183
898 622 182
898 620 185
902 626 184
920 628 185
918 662 162
902 608 221
934 662 188
924 672 184
932 670 200
926 682 192
924 688 190
924 692 189
922 690 190
912 674 204
912 678 196
916 684 186
920 686 180
916 666 191
922 672 180
934 678 164
942 676 165
926 674 162
926 668 161
922 658 166
1424 548 232
922 650 161
928 648 162
934 650 163
1586 518 249
1466 418 216
958 672 158
908 630 210
1486 430 243
940 666 182
1382 532 249
944 682 172
940 682 177
932 662 197
928 672 196
926 686 189
930 690 184
926 702 180
1428 448 233
922 702 188
926 704 192
922 698 195
924 702 193
924 706 192
920 700 199
920 694 205
918 696 205
920 702 201
914 684 218
918 700 204
926 702 211
912 680 224
918 694 210
914 672 232
916 690 214
916 692 212
918 694 211
912 684 222
914 694 212
918 696 209
910 684 222
916 698 209
916 698 209
920 700 215
908 666 242
908 686 222
908 668 240
908 690 218
906 684 2

1920.0